# 1. Data Extraction
## Extracción de los datos que están en el contenedor del Storage Account de Microsoft Azure.

### Primero definimos las variables necesarias para ingresar al contenedor

In [0]:
storage_account_name = "proyecto123"
storage_account_access_key = "gDBiZp1U9O04TBWw75JlQ10RyoE4BA2req7X5QkVNTidivJkstpvYpHJwilPUodqMfucp89uQUw7+AStRH5msw=="
container = "datos"
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",storage_account_access_key)
# Listamos el path para cada archivo junto con su nombre y demás propiedades
files_raw = dbutils.fs.ls(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/")

In [0]:
# A partir de la lista de dbutils realizamos una nueva lista donde obtenemos las URLs para cada archivo
file_list = [str(i).split('\'')[1] for i in files_raw]
# Eliminamos los datos de metadata -> Generan problemas en la ejecución
metadata = file_list.pop()
# A partir de file_list también podemos realizar una nueva lista que nos permita obtener los nombres de los archivos
file_names = [i.split('/')[3] for i in file_list]

### Creamos una función que reciba el path del archivo y nos genere el dataframe

In [0]:
def get_df(file_location):
    file_type = "json"
    spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)
    df = spark.read.format(file_type).option("inferSchema", "true").load(file_location)
    return df

### Creamos los dataframes a partir  de la función previamente definida

In [0]:
# Creating dataframes
amazon_instant_video = get_df(file_list[0])
apps_for_android = get_df(file_list[1])
automotive = get_df(file_list[2])
baby = get_df(file_list[3])
beauty = get_df(file_list[4])
books = get_df(file_list[5])
cds_and_vinyl = get_df(file_list[6])
cell_phones_accessories = get_df(file_list[7])
clothing_shoes_jewelry = get_df(file_list[8])
digital_music = get_df(file_list[9])
electronics = get_df(file_list[10])
grocery_and_gourmet_food = get_df(file_list[11])
health_and_personal = get_df(file_list[12])
home_and_kitchen = get_df(file_list[13])
kindle_store = get_df(file_list[14])
movies_and_tv = get_df(file_list[15])
musical_instruments = get_df(file_list[16])
office_products = get_df(file_list[17])
patio_lawn_garden = get_df(file_list[18])
pet_supplies = get_df(file_list[19])
sports_and_outdoors = get_df(file_list[20])
toys_and_games = get_df(file_list[21])
videogames = get_df(file_list[22])

# 2. Transformación
## Continuamos con la transformación de nuestros dataframes. Afortunadamente, comparten las mismas columnas

In [0]:
# Uniendo los dataframes
from pyspark.sql import DataFrame
from functools import reduce
def unionAll(*dfs):
    df_union = reduce(DataFrame.unionAll, dfs)
    return df_union
df_complete = unionAll(amazon_instant_video, apps_for_android, automotive, baby, beauty, books, cds_and_vinyl, cell_phones_accessories, clothing_shoes_jewelry,
              digital_music, electronics, grocery_and_gourmet_food, health_and_personal, home_and_kitchen, kindle_store, movies_and_tv, musical_instruments, office_products,patio_lawn_garden,                         pet_supplies, sports_and_outdoors, toys_and_games, videogames)

In [0]:
# Shape
print(df_complete.count(), len(df_complete.columns))

17954953 9

In [0]:
# Cambiando nombre de columnas y dropeando la columna de reviewTime
df_complete = df_complete.withColumnRenamed("asin", "asinID")
df_complete = df_complete.drop('reviewTime')

In [0]:
# Cambiando la columna de Helpful -> Creando dos nuevas columnas que separen los datos
from pyspark.sql.functions import expr
df_complete = df_complete.select(["asinID", "reviewText", "reviewerID", "reviewerName", "summary", "unixReviewTime", "helpful"]+[expr("helpful[" + str(x)+ "]") for x in range(0, 2)]).drop('helpful')

In [0]:
# Cambiando UnixTime a datetime para mejorar el filtrado
from pyspark.sql.functions import from_unixtime
df_complete = df_complete.withColumn("datetime", from_unixtime("unixReviewTime")).drop("UnixReviewTime")

In [0]:
# Organizando 
df_complete = df_complete.orderBy("datetime", ascending = False)

In [0]:
df_complete.show()

+----------+--------------------+--------------+--------------------+--------------------+----------+----------+-------------------+
 asinID| reviewText| reviewerID| reviewerName| summary|helpful[0]|helpful[1]| datetime|
+----------+--------------------+--------------+--------------------+--------------------+----------+----------+-------------------+
B000H00VBQ|I had big expecta...|A11N155CW1UV02| AdrianaM|A little bit bori...| 0| 0|2014-05-03 00:00:00|
B000H00VBQ|I highly recommen...|A3BC8O2KCL29V2| Carol T|Excellent Grown U...| 0| 0|2012-09-03 00:00:00|
B000H00VBQ|This one is a rea...| A60D5HQFOTSOM|Daniel Cooper "da...|Way too boring fo...| 0| 1|2013-10-16 00:00:00|
B000H00VBQ|Mysteries are int...|A1RJPIGRSNX4PW| J. Kaplan "JJ"|Robson Green is m...| 0| 0|2013-10-30 00:00:00|
B000H00VBQ|This show always ...|A16XRPF40679KG| Michael Dobey|Robson green and ...| 1| 1|2009-02-11 00:00:00|
B000H00VBQ|I discovered this...|A1POFVVXUZR3IQ| Z Hayes|I purchased the s...| 12| 12|2011-10-11 00:00:00|
B000H0X79O|It beats watching...|A1PG2VV4W1WRPL|Jimmy C. Saunders...|It takes up your ...| 0| 0|2013-10-15 00:00:00|
B000H0X79O|There are many ep...| ATASGS8HZHGIB| JohnnyC|A reasonable way ...| 0| 0|2013-12-29 00:00:00|
B000H0X79O|This is the best ...|A3RXD7Z44T9DHW| Kansas| kansas001| 0| 0|2014-02-26 00:00:00|
B000H0X79O|Not bad. Didn't ...| AUX8EUBNTHIIU| Louis V. Borsellino| Entertaining Comedy| 0| 0|2014-04-02 00:00:00|
B000H0X79O|Funny, interestin...| AXM3GQLD0CHIL| Ray Shiva| Worth watching!| 0| 0|2014-02-07 00:00:00|
B000H29TXU|I love the variet...|A398QSASJOIKA6| Amazon Customer|comedy club quali...| 0| 0|2014-02-06 00:00:00|
B000H29TXU|comedy is a matte...|A2U61O0KWJH3MM| Cathy P.| ok| 0| 1|2013-09-05 00:00:00|
B000H29TXU|if this had to do...|A2LSZFEFTDRDIJ| debra marrero|not sure who this...| 0| 0|2013-06-14 00:00:00|
B000H29TXU|Watched it for Ke...|A39F2EW27YYUDM| Emily Booth| Loved it!| 0| 0|2014-04-29 00:00:00|
B000H29TXU|he's OK. His humo...| AO1Z63O9CTT95| James Westervelt|same routine he d...| 0| 0|2014-03-06 00:00:00|
B000H2DMME|some comedians ar...|A1378ZJMWCTVT3| Alla| it was ok| 0| 0|2014-01-09 00:00:00|
B000H2DMME|I only watched th...|A2JE0JYIZL5NU4| C. A. Neal|I Only Watched Wa...| 0| 0|2014-05-18 00:00:00|
B000H2DMME|Enjoyed some of t...|A2LSZFEFTDRDIJ| debra marrero| Some were funny| 0| 0|2013-02-10 00:00:00|
B000H2DMME|All the comedians...|A3RXD7Z44T9DHW| Kansas| kansas| 0| 0|2014-03-06 00:00:00|
+----------+--------------------+--------------+--------------------+--------------------+----------+----------+-------------------+
only showing top 20 rows

# 3. Load: Creando la base de datos con nuestros datos transformados

In [0]:
# Agregando la tabla a la base de datos SQL de databricks
df_complete.write.format("parquet").saveAsTable("reviews")

In [0]:
def update_database(df):
    
    '''Esta función recibe un dataframe de spark y exporta los datos a la base de datos de Azure SQL'''
    
    jdbcHostname = "skaivudbserver.database.windows.net"
    jdbcPort = "1433"
    jdbcDatabase = "skaivuinsights_db"
    properties = {"user": "awsprojectskaivu", "password": "Skaivuinsights1"}
    
    url = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname,jdbcPort,jdbcDatabase)
    
    df = DataFrameWriter(df)
    return df.jdbc(url = url, table = "reviews", mode = "overwrite", properties = properties)

In [0]:
# Generando querys para hacer la ingesta delta
# YEARS: 2010 - 2014
df_2010_2014 = spark.sql("SELECT * FROM reviews WHERE YEAR(datetime) BETWEEN '2010' AND '2014' ")

In [0]:
update_database(df_2010_2014)

In [0]:
df_2007_2009 = spark.sql("SELECT * FROM reviews WHERE YEAR(datetime) BETWEEN '2010' AND '2014' ")

In [0]:
df_2004_2006 = spark.sql("SELECT * FROM reviews WHERE YEAR(datetime) BETWEEN '2004' AND '2006' ")

In [0]:
df_2000_2003 = spark.sql("SELECT * FROM reviews WHERE YEAR(datetime) BETWEEN '2010' AND '2014' ")